In [ ]:
from IPython.display import display, clear_output
import ipywidgets as ipw
import widgets
import utils
import os
import json
import shutil

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
slabs_options = [object_key for object_key, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab"]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

material_selector = widgets.MaterialSelectionWidget()

molecule_selector_list = widgets.MultipleSelectorWidget("molecule")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule', disabled = False, button_style = '', 
    tooltip = 'Add molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule', disabled = False, button_style = '', 
    tooltip = 'Remove molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

# chemist_selector = widgets.ChemistSelectionWidget()
# chemist_selector.load_dropdown_box()

# storage_selector = widgets.StorageSelectionWidget()
# storage_selector.load_dropdown_box()

# supplier_selector = widgets.SupplierSelectionWidget()
# supplier_selector.load_dropdown_box()

product_widgets = widgets.ObjectPropertiesWidgets("Reaction Product")

product_support_files = ipw.FileUpload(multiple = True)

product_cdxml = ipw.FileUpload(multiple = False, accept = '.cdxml')

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_product_openbis(b):
    if not product_cdxml.value:
        display(utils.Javascript(data = "alert('Upload a CDXML file.')"))
        return
    
    product_parents = []
    if molecule_selector_list.selectors:
        for molecule_selector in molecule_selector_list.selectors:
            if molecule_selector.dropdown.value != -1:
                product_parents.append(molecule_selector.dropdown.value)
                
    if material_selector.dropdown.value != -1:
        product_parents.append(material_selector.dropdown.value)
        
    object_properties = {}
    for prop in CONFIG["objects"]["Reaction Product"]["properties"]:
        if prop == "yield":
            if product_widgets.properties_widgets[prop].value == 0:
                continue
            
        if CONFIG["properties"][prop]["property_type"] == "QUANTITY_VALUE":
            object_properties[prop] = json.dumps(
                {
                    "has_value": product_widgets.properties_widgets[prop].children[0].value, 
                    "has_unit": product_widgets.properties_widgets[prop].children[1].value
                }
            )
        else: 
            object_properties[prop] = product_widgets.properties_widgets[prop].value

    # Check if the product is already in openBIS
    products_openbis = [f"{product.props['$name']}" for product in utils.get_openbis_objects(OPENBIS_SESSION, type ="REACTION_PRODUCT")]
    
    product_name = object_properties['$name']
    if product_name in products_openbis:
        display(utils.Javascript(data = "alert('Reaction product is already in openBIS!')"))
        return

    product_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="REACTION_PRODUCT", 
        collection="/MATERIALS/SUBSTANCES/REACTION_PRODUCT_COLLECTION", 
        props = object_properties,
        parents = product_parents
    )
    
    upload_datasets(product_object, product_cdxml)
    
    upload_datasets(product_object, product_support_files)
    
    print("Upload successful!")

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

def checkbox_on_change(checkbox, textbox):
    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True
    checkbox.observe(enable_textbox, names = "value")

def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def select_material_radio_change(change):
    material_type = material_selection_radio_button.value
    if material_type == "No material":
        with material_selector:
            clear_output()
            return
    
    material_selector.details_textbox.value = ''
    material_types = {}
    for object_key, object_info in CONFIG["objects"].items():
        if object_info["object_type"] == "slab":
            material_types[object_key] = (object_info["openbis_object_type"], object_info["placeholder"])
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_class, placeholder = material_types.get(material_type)
        material_selector.load_dropdown_box(material_class, placeholder)
        display(ipw.VBox(display_list))

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    if material_selector.dropdown_boxes.children[0].value == -1:
        material_selector.details_textbox.value = ''
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])
        return
    
    # Get selected object properties information from config file
    selected_object = material_selection_radio_button.value
    selected_object_properties = CONFIG["objects"][selected_object]["properties"]
    
    # Get material object information and dataset
    material_object = OPENBIS_SESSION.get_object(material_selector.dropdown_boxes.children[0].value)
    material_dataset = material_object.get_datasets()[0]
    
    # Get the object image preview
    if material_dataset:
        material_dataset.download(destination="images")
        material_selector.image_box.value = utils.read_file(f"images/{material_dataset.permId}/{material_dataset.file_list[0]}")
        # Erase file after downloading it
        shutil.rmtree(f"images/{material_dataset.permId}")
    else:
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])

    # Make a string with the property values of the object
    material_metadata = material_object.props.all()
    material_metadata_string = ""
    for prop_key in selected_object_properties:
        prop_title = CONFIG["properties"][prop_key]["title"]
        if CONFIG["properties"][prop_key]["property_type"] == "QUANTITY_VALUE":
            value = material_metadata.get(prop_key)
            if value:
                prop_dict = json.loads(value)
                material_metadata_string += f"{prop_title}: {prop_dict['value']} {prop_dict['unit']}\n"
            else:
                material_metadata_string += f"{prop_title}: {value}\n"
        else:
            material_metadata_string += f"{prop_title}: {material_metadata.get(prop_key)}\n"

    material_selector.details_textbox.value = material_metadata_string


# Create reaction product

## Select molecule

In [ ]:
display(molecule_selector_list_output)
with molecule_selector_list_output:
    display(molecule_selector_list)  
display(add_remove_molecule_buttons_hbox)

## Select slab

In [ ]:
display(material_selection_radio_button)
display(material_selector)

## Properties

In [ ]:
display(product_widgets)

## Upload CDXML

In [ ]:
display(product_cdxml)

## Support files

In [ ]:
display(product_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(load_material_metadata, names = 'value')
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
create_button.on_click(create_product_openbis)
quit_button.on_click(close_notebook)